<a href="https://colab.research.google.com/github/blacktruth513/CodeStates_Project/blob/main/Section2_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d ruiqurm/lianjia
#!ls
!unzip lianjia.zip 

Saving kaggle.json to kaggle (3).json
lianjia.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  lianjia.zip
replace new.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [2]:
# %%capture
# import sys

# if 'google.colab' in sys.modules:
#     # Install packages in Colab
#     !pip install category_encoders==2.*
#     !pip install eli5
#     !pip install pandas-profiling==2.*
#     !pip install pdpbox
#     !pip install shap

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('./new.csv', encoding = 'GB2312',low_memory=False)  #GB/T 2312-1980 is a key official character set of the People's Republic of China, used for Simplified Chinese characters.
df

,url,id,Lng,Lat,Cid,tradeTime,DOM,followers,totalPrice,price,square,livingRoom,drawingRoom,kitchen,bathRoom,floor,buildingType,constructionTime,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage
0,https://bj.lianjia.com/chengjiao/101084782030....,101084782030,116.475489,40.019520,1111027376244,2016-08-09,1464.0,106,415.0,31680,131.00,2,1,1,1,高 26,1.0,2005,3,6,0.217,1.0,0.0,1.0,7,56021.0
1,https://bj.lianjia.com/chengjiao/101086012217....,101086012217,116.453917,39.881534,1111027381879,2016-07-28,903.0,126,575.0,43436,132.38,2,2,1,2,高 22,1.0,2004,4,6,0.667,1.0,1.0,0.0,7,71539.0
2,https://bj.lianjia.com/chengjiao/101086041636....,101086041636,116.561978,39.877145,1111040862969,2016-12-11,1271.0,48,1030.0,52021,198.00,3,2,1,3,中 4,4.0,2005,3,6,0.500,1.0,0.0,0.0,7,48160.0
3,https://bj.lianjia.com/chengjiao/101086406841....,101086406841,116.438010,40.076114,1111043185817,2016-09-30,965.0,138,297.5,22202,134.00,3,1,1,1,底 21,1.0,2008,1,6,0.273,1.0,0.0,0.0,6,51238.0
4,https://bj.lianjia.com/chengjiao/101086920653....,101086920653,116.428392,39.886229,1111027381174,2016-08-28,927.0,286,392.0,48396,81.00,2,1,1,1,中 6,4.0,1960,2,2,0.333,0.0,1.0,1.0,1,62588.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318846,https://bj.lianjia.com/chengjiao/BJYZ92311192....,BJYZ92311192,116.546899,39.755236,1111027377053,2016-06-10,1.0,8,350.0,20580,170.07,3,2,1,2,中 5,4.0,2003,1,6,0.333,0.0,1.0,0.0,3,36545.0
318847,https://bj.lianjia.com/chengjiao/BJYZ92320171....,BJYZ92320171,116.497474,39.810115,1111027377957,2016-06-10,NaN,1,108.8,31006,35.09,1,0,1,1,中 24,1.0,2009,1,6,0.222,1.0,0.0,1.0,3,54282.0
318848,https://bj.lianjia.com/chengjiao/BJYZ92324217....,BJYZ92324217,116.497256,39.804081,1111027380056,2016-06-05,1.0,2,359.0,35138,102.17,2,2,1,2,中 7,4.0,2000,3,6,0.500,1.0,1.0,0.0,3,46927.0
318849,https://bj.lianjia.com/chengjiao/BJYZ92333313....,BJYZ92333313,116.501794,39.799347,1111027377054,2016-06-12,NaN,4,720.0,40373,178.34,4,2,1,2,中 5,4.0,2003,4,2,0.500,0.0,1.0,0.0,3,54842.0


In [12]:
#실험용 데이터 카피 
df_copy = df.copy()
df_copy

,url,id,Lng,Lat,Cid,tradeTime,DOM,followers,totalPrice,price,square,livingRoom,drawingRoom,kitchen,bathRoom,floor,buildingType,constructionTime,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage
0,https://bj.lianjia.com/chengjiao/101084782030....,101084782030,116.475489,40.019520,1111027376244,2016-08-09,1464.0,106,415.0,31680,131.00,2,1,1,1,高 26,1.0,2005,3,6,0.217,1.0,0.0,1.0,7,56021.0
1,https://bj.lianjia.com/chengjiao/101086012217....,101086012217,116.453917,39.881534,1111027381879,2016-07-28,903.0,126,575.0,43436,132.38,2,2,1,2,高 22,1.0,2004,4,6,0.667,1.0,1.0,0.0,7,71539.0
2,https://bj.lianjia.com/chengjiao/101086041636....,101086041636,116.561978,39.877145,1111040862969,2016-12-11,1271.0,48,1030.0,52021,198.00,3,2,1,3,中 4,4.0,2005,3,6,0.500,1.0,0.0,0.0,7,48160.0
3,https://bj.lianjia.com/chengjiao/101086406841....,101086406841,116.438010,40.076114,1111043185817,2016-09-30,965.0,138,297.5,22202,134.00,3,1,1,1,底 21,1.0,2008,1,6,0.273,1.0,0.0,0.0,6,51238.0
4,https://bj.lianjia.com/chengjiao/101086920653....,101086920653,116.428392,39.886229,1111027381174,2016-08-28,927.0,286,392.0,48396,81.00,2,1,1,1,中 6,4.0,1960,2,2,0.333,0.0,1.0,1.0,1,62588.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318846,https://bj.lianjia.com/chengjiao/BJYZ92311192....,BJYZ92311192,116.546899,39.755236,1111027377053,2016-06-10,1.0,8,350.0,20580,170.07,3,2,1,2,中 5,4.0,2003,1,6,0.333,0.0,1.0,0.0,3,36545.0
318847,https://bj.lianjia.com/chengjiao/BJYZ92320171....,BJYZ92320171,116.497474,39.810115,1111027377957,2016-06-10,NaN,1,108.8,31006,35.09,1,0,1,1,中 24,1.0,2009,1,6,0.222,1.0,0.0,1.0,3,54282.0
318848,https://bj.lianjia.com/chengjiao/BJYZ92324217....,BJYZ92324217,116.497256,39.804081,1111027380056,2016-06-05,1.0,2,359.0,35138,102.17,2,2,1,2,中 7,4.0,2000,3,6,0.500,1.0,1.0,0.0,3,46927.0
318849,https://bj.lianjia.com/chengjiao/BJYZ92333313....,BJYZ92333313,116.501794,39.799347,1111027377054,2016-06-12,NaN,4,720.0,40373,178.34,4,2,1,2,中 5,4.0,2003,4,2,0.500,0.0,1.0,0.0,3,54842.0


# 데이터 설명

- 链家网是集房源信息搜索、产品研发、大数据处理、服务标准建立为一体的以数据驱动的全价值链房产服务平台。主营：二手房、租房、新房。您可通过小区找房，链家App等找房。 <br> 
:  "리엔지아"는 한국의 다방, 직방과 같은 중국의 대표적인 판매·임대·분양용 주택 공급 플랫폼으로 주요 영업(판매) 상품은 중고 주택, 새 주택, 렌트 등이 있다.

- Contents : Housing price of Beijing from 2011 to 2017, fetching from Lianjia.com
- Inspiration : 
It may help you predict the housing price of Beijing.

#변수 설명

- url: the url which fetches the data
-id: the id of transaction
-Lng: and Lat coordinates, using the BD09 protocol.
-Cid: community id
-tradeTime: the time of transaction
-DOM: active days on market.Know more in https://en.wikipedia.org/wiki/ / Days_on_market / 부동산이 매물로 시장에 나와 있는 기간
-followers: the number of people follow the transaction. / 거래 참여자 수
-totalPrice: the total price
-price: the average price by square
-square: the square of house
-livingRoom: the number of living room
-drawingRoom: the number of drawing room
-kitchen: the number of kitchen
-bathroom the number of bathroom
-floor: the height of the house. I will turn the Chinese characters to English in the next version.
-buildingType: including tower( 1 ) , bungalow( 2 )，combination of plate and tower( 3 ), plate( 4 ). / 범주형 
-constructionTime: the time of construction
-renovationCondition: including other( 1 ), rough( 2 ),Simplicity( 3 ), hardcover( 4 ) / 범주형
-buildingStructure: including unknow( 1 ), mixed( 2 ), brick and wood( 3 ), brick and concrete( 4 ),steel( 5 ) and steel-concrete composite ( 6 ). / 범주형
-ladderRatio: the proportion between number of residents on the same floor and number of elevator of ladder. It describes how many ladders a resident have on average.
-elevator have ( 1 ) or not have elevator( 0 ) / 범주형
-fiveYearsProperty: It's related to China restricted purchase of houses policy / 범주형

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318851 entries, 0 to 318850
Data columns (total 26 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   url                  318851 non-null  object 
 1   id                   318851 non-null  object 
 2   Lng                  318851 non-null  float64
 3   Lat                  318851 non-null  float64
 4   Cid                  318851 non-null  int64  
 5   tradeTime            318851 non-null  object 
 6   DOM                  160874 non-null  float64
 7   followers            318851 non-null  int64  
 8   totalPrice           318851 non-null  float64
 9   price                318851 non-null  int64  
 10  square               318851 non-null  float64
 11  livingRoom           318851 non-null  object 
 12  drawingRoom          318851 non-null  object 
 13  kitchen              318851 non-null  int64  
 14  bathRoom             318851 non-null  object 
 15  floor            

In [24]:
df.describe()

,Lng,Lat,Cid,DOM,followers,totalPrice,price,square,kitchen,buildingType,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage
count,318851.000000,318851.000000,3.188510e+05,160874.000000,318851.000000,318851.000000,318851.000000,318851.000000,318851.000000,316830.000000,318851.000000,318851.000000,3.188510e+05,318819.000000,318819.000000,318819.000000,318851.000000,318388.000000
mean,116.418459,39.949591,1.129113e+12,28.822339,16.731508,349.030201,43530.436379,83.240597,0.994599,3.009790,2.606330,4.451026,6.316486e+01,0.577055,0.645601,0.601112,6.763564,63682.446305
std,0.112054,0.091983,2.363447e+12,50.237343,34.209185,230.780778,21709.024204,37.234661,0.109609,1.269857,1.311739,1.901753,2.506851e+04,0.494028,0.478331,0.489670,2.812616,22329.215447
min,116.072514,39.627030,1.111027e+12,1.000000,0.000000,0.100000,1.000000,6.900000,0.000000,0.048000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,1.000000,10847.000000
25%,116.344985,39.893200,1.111027e+12,1.000000,0.000000,205.000000,28050.000000,57.900000,1.000000,1.000000,1.000000,2.000000,2.500000e-01,0.000000,0.000000,0.000000,6.000000,46339.000000
50%,116.416780,39.934527,1.111027e+12,6.000000,5.000000,294.000000,38737.000000,74.260000,1.000000,4.000000,3.000000,6.000000,3.330000e-01,1.000000,1.000000,1.000000,7.000000,59015.000000
75%,116.477581,40.003018,1.111027e+12,37.000000,18.000000,425.500000,53819.500000,98.710000,1.000000,4.000000,4.000000,6.000000,5.000000e-01,1.000000,1.000000,1.000000,8.000000,75950.000000
max,116.732378,40.252758,1.114620e+15,1677.000000,1143.000000,18130.000000,156250.000000,1745.500000,4.000000,4.000000,4.000000,6.000000,1.000940e+07,1.000000,1.000000,1.000000,13.000000,183109.000000


In [ ]:
# 데이터 결측값 확인
df.isnull().sum()

url                         0
id                          0
Lng                         0
Lat                         0
Cid                         0
tradeTime                   0
DOM                    157977
followers                   0
totalPrice                  0
price                       0
square                      0
livingRoom                  0
drawingRoom                 0
kitchen                     0
bathRoom                    0
floor                       0
buildingType             2021
constructionTime            0
renovationCondition         0
buildingStructure           0
ladderRatio                 0
elevator                   32
fiveYearsProperty          32
subway                     32
district                    0
communityAverage          463
dtype: int64

In [18]:
# 특성 간 상관 관계 분석 (수치형 변수) 
corr_df1 = df.corr(method='pearson') 
corr_df1

,Lng,Lat,Cid,DOM,followers,totalPrice,price,square,livingRoom,drawingRoom,kitchen,bathRoom,floor,buildingType,constructionTime,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage
Lng,1.000000,0.045791,-0.009963,-0.009812,-0.010010,-0.073584,-0.154205,0.065473,-0.010645,0.074175,0.072081,0.060581,0.025828,0.014602,0.097244,-0.008179,0.035648,0.002110,0.039764,0.014476,-0.016897,0.217120,-0.178480
Lat,0.045791,1.000000,-0.005625,0.016349,-0.007436,0.014395,-0.055544,0.120405,0.085276,0.052976,-0.002922,0.090835,-0.092479,0.097136,0.033688,0.011224,-0.155062,-0.000210,-0.100141,0.004772,-0.085856,0.316810,-0.073118
Cid,-0.009963,-0.005625,1.000000,0.003120,0.003150,0.002120,0.000230,0.001148,0.009022,0.003617,-0.002242,0.003381,0.011068,0.006220,0.015359,0.001802,0.006397,-0.000030,0.002806,-0.012573,0.005676,-0.007007,-0.007787
DOM,-0.009812,0.016349,0.003120,1.000000,0.426924,0.186567,0.171787,0.056980,0.038963,0.004298,0.009896,0.043174,0.020541,-0.010739,0.024001,0.095705,0.021693,-0.000807,0.022163,-0.034108,0.009758,0.000129,0.022476
followers,-0.010010,-0.007436,0.003150,0.426924,1.000000,0.160809,0.261171,-0.051161,-0.008681,-0.056454,0.008732,-0.034176,-0.012391,0.006868,-0.019940,0.242336,-0.018465,-0.000237,-0.015979,0.045699,0.020673,-0.030759,-0.009676
totalPrice,-0.073584,0.014395,0.002120,0.186567,0.160809,1.000000,0.643434,0.559300,0.426858,0.314819,0.084653,0.430233,0.167659,-0.063662,0.123105,0.260409,0.196989,-0.001134,0.209454,-0.009499,0.116821,0.012772,0.432971
price,-0.154205,-0.055544,0.000230,0.171787,0.261171,0.643434,1.000000,-0.167953,-0.073659,-0.127205,0.033190,-0.079405,0.028907,-0.053734,-0.209220,0.299273,0.046783,0.001107,0.053377,0.007031,0.230399,0.037682,0.682431
square,0.065473,0.120405,0.001148,0.056980,-0.051161,0.559300,-0.167953,1.000000,0.720008,0.616475,0.080377,0.731341,0.161586,-0.014472,0.367524,0.032367,0.164099,-0.002570,0.191965,-0.020013,-0.095064,-0.006404,-0.149905
livingRoom,-0.010645,0.085276,0.009022,0.038963,-0.008681,0.426858,-0.073659,0.720008,1.000000,0.476354,0.093157,0.538575,-0.085417,0.118706,-0.001932,0.008229,-0.062932,-0.003425,-0.067015,0.043295,-0.059670,0.018101,-0.052558
drawingRoom,0.074175,0.052976,0.003617,0.004298,-0.056454,0.314819,-0.127205,0.616475,0.476354,1.000000,0.122823,0.521730,0.034815,0.074378,0.211353,-0.010563,0.057008,-0.000865,0.056290,0.005198,-0.094270,0.011553,-0.095206


In [19]:
# 특성 간 상관 관계 분석 (수치형 변수)
corr_df1 = corr_df1.loc['totalPrice']
corr_df1 = corr_df1.drop(['totalPrice']) #동일 변수 상관계수 drop

In [20]:
corr_top = pd.DataFrame(corr_df1.sort_values(ascending=False)) #시리즈 > DF化
corr_top

,totalPrice
price,0.643434
square,0.559300
communityAverage,0.432971
bathRoom,0.430233
livingRoom,0.426858
drawingRoom,0.314819
renovationCondition,0.260409
elevator,0.209454
buildingStructure,0.196989
DOM,0.186567


- 상관관계 분석 결과 : price - square - communityAverage 순으로 높은 상관관계를 보이고 있으며 상관관계가 상대적으로 낮은 변수들에 대해 분석에 사용 여부를 정할 필요성이 있다고 판단했다. / 분석 요소 : target 변수인 totalprice에 대해 price 변수는 totalprice을 square 값으로 나눠 만든 변수이므로 상관관계와 인과관계에 대해 생각해봐야한다. 

In [4]:
################################################# 데이터 전처리 ####################################################### 

# 1) 변수에 포함되어 있는 한자 등 특수 문자 제거 ex) (高 ，中， 底 : 주택의 높이를 나타내는 한자)
df['floor'] = df['floor'].replace(r'[^0-9a-zA-Z ]', '', regex=True).replace("'", '') # 정규식 활용

# 2) 알 수 없는 관측치 제거  
#df['constructionTime'].value_counts() # 약 19,000개의 건축 연도를 알 수 없는 데이터 값이 있다. 데이터 수를 고려했을 때, 이 값들을 제거해도 된다고 판단 
df = df[~df['constructionTime'].isin(['未知'])]

# 3) 날짜형 컬럼 형 변환
df['tradeTime'] = pd.to_datetime(df['tradeTime'])

# 4) 컬럼에 포함되어 있는 결측값을 적절한 값 대체(DOM 컬럼에 대하여)
# Days_on_market(DOM)란, 부동산이 매물로 시장에 나와 있는 기간(단위 / 일)로서, 본 데이터에 포함되어 있는 해당 컬럼의 결측치가 약 15만개(데이터 절반)에 달해 이 값을 평균값으로 대체하여 연구에 사용할 수 있도록 했다.
df['DOM'].fillna(df['DOM'].mean(),inplace=True) 

# 5) 결측치 제거 / 나머지 컬럼의 결측값이 비교적 적으므로 영향이 적다고 판단하여 결측치를 제거
df = df.dropna(axis=0, how='any')

# 6) object형 자료형을 가진 컬럼에 대해 int형으로 형 변환
df['livingRoom'] = df['livingRoom'].astype('int64')
df['drawingRoom']= df['drawingRoom'].astype('int64')
df['bathRoom']= df['bathRoom'].astype('int64')
df['constructionTime'] = df['constructionTime'].astype('int64')
df['buildingType']= df['buildingType'].astype('int64') 
df['floor']= df['floor'].astype('int64') 

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297701 entries, 0 to 318849
Data columns (total 26 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   url                  297701 non-null  object        
 1   id                   297701 non-null  object        
 2   Lng                  297701 non-null  float64       
 3   Lat                  297701 non-null  float64       
 4   Cid                  297701 non-null  int64         
 5   tradeTime            297701 non-null  datetime64[ns]
 6   DOM                  297701 non-null  float64       
 7   followers            297701 non-null  int64         
 8   totalPrice           297701 non-null  float64       
 9   price                297701 non-null  int64         
 10  square               297701 non-null  float64       
 11  livingRoom           297701 non-null  int64         
 12  drawingRoom          297701 non-null  int64         
 13  kitchen       

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299568 entries, 0 to 318849
Data columns (total 26 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   url                  299568 non-null  object        
 1   id                   299568 non-null  object        
 2   Lng                  299568 non-null  float64       
 3   Lat                  299568 non-null  float64       
 4   Cid                  299568 non-null  int64         
 5   tradeTime            299568 non-null  datetime64[ns]
 6   DOM                  299568 non-null  float64       
 7   followers            299568 non-null  int64         
 8   totalPrice           299568 non-null  float64       
 9   price                299568 non-null  int64         
 10  square               299568 non-null  float64       
 11  livingRoom           299568 non-null  object        
 12  drawingRoom          299568 non-null  object        
 13  kitchen       

In [17]:
df

,url,id,Lng,Lat,Cid,tradeTime,DOM,followers,totalPrice,price,square,livingRoom,drawingRoom,kitchen,bathRoom,floor,buildingType,constructionTime,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage
0,https://bj.lianjia.com/chengjiao/101084782030....,101084782030,116.475489,40.019520,1111027376244,2016-08-09,1464.000000,106,415.0,31680,131.00,2,1,1,1,26,1,2005,3,6,0.217,1.0,0.0,1.0,7,56021.0
1,https://bj.lianjia.com/chengjiao/101086012217....,101086012217,116.453917,39.881534,1111027381879,2016-07-28,903.000000,126,575.0,43436,132.38,2,2,1,2,22,1,2004,4,6,0.667,1.0,1.0,0.0,7,71539.0
2,https://bj.lianjia.com/chengjiao/101086041636....,101086041636,116.561978,39.877145,1111040862969,2016-12-11,1271.000000,48,1030.0,52021,198.00,3,2,1,3,4,4,2005,3,6,0.500,1.0,0.0,0.0,7,48160.0
3,https://bj.lianjia.com/chengjiao/101086406841....,101086406841,116.438010,40.076114,1111043185817,2016-09-30,965.000000,138,297.5,22202,134.00,3,1,1,1,21,1,2008,1,6,0.273,1.0,0.0,0.0,6,51238.0
4,https://bj.lianjia.com/chengjiao/101086920653....,101086920653,116.428392,39.886229,1111027381174,2016-08-28,927.000000,286,392.0,48396,81.00,2,1,1,1,6,4,1960,2,2,0.333,0.0,1.0,1.0,1,62588.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318845,https://bj.lianjia.com/chengjiao/BJYZ92303482....,BJYZ92303482,116.546899,39.755236,1111027377053,2016-06-09,1.000000,8,355.5,26491,134.20,3,2,1,2,9,3,2009,4,6,0.500,1.0,1.0,0.0,3,36545.0
318846,https://bj.lianjia.com/chengjiao/BJYZ92311192....,BJYZ92311192,116.546899,39.755236,1111027377053,2016-06-10,1.000000,8,350.0,20580,170.07,3,2,1,2,5,4,2003,1,6,0.333,0.0,1.0,0.0,3,36545.0
318847,https://bj.lianjia.com/chengjiao/BJYZ92320171....,BJYZ92320171,116.497474,39.810115,1111027377957,2016-06-10,29.527079,1,108.8,31006,35.09,1,0,1,1,24,1,2009,1,6,0.222,1.0,0.0,1.0,3,54282.0
318848,https://bj.lianjia.com/chengjiao/BJYZ92324217....,BJYZ92324217,116.497256,39.804081,1111027380056,2016-06-05,1.000000,2,359.0,35138,102.17,2,2,1,2,7,4,2000,3,6,0.500,1.0,1.0,0.0,3,46927.0
